In [14]:
from optimize import optimize
import pandas as pd
import seaborn as sns

### Load Data

In [ ]:
## commodity prices ##
commodities = pd.read_csv('../Data/raw/commodity_prices.csv',index_col=['Date'])
commodities.index = pd.to_datetime(commodities.index)

## stock prices ##
stocks = pd.read_csv('../Data/raw/stock_prices.csv',index_col=['Date'])
stocks.index = pd.to_datetime(stocks.index)

### Test optimizer

In [7]:

# Example pair
commodity = commodities['SI=F Close']
stock = stocks['SNAP']

param_grid = {
    'threshold': [0.3, 0.80, 0.85, 0.90, 0.95],
    'streak': [2, 3, 5],
    'quantile': [0.3, 0.4, 0.5, 0.6]
}

results = optimize(commodity, stock, param_grid)
print(results)

    threshold  streak  quantile  final_return
30       0.85       3       0.5      1.000000
45       0.90       5       0.4      1.000000
33       0.85       5       0.4      1.000000
34       0.85       5       0.5      1.000000
35       0.85       5       0.6      1.000000
36       0.90       2       0.3      1.000000
37       0.90       2       0.4      1.000000
38       0.90       2       0.5      1.000000
39       0.90       2       0.6      1.000000
40       0.90       3       0.3      1.000000
41       0.90       3       0.4      1.000000
42       0.90       3       0.5      1.000000
43       0.90       3       0.6      1.000000
44       0.90       5       0.3      1.000000
46       0.90       5       0.5      1.000000
31       0.85       3       0.6      1.000000
47       0.90       5       0.6      1.000000
48       0.95       2       0.3      1.000000
49       0.95       2       0.4      1.000000
50       0.95       2       0.5      1.000000
51       0.95       2       0.6   